## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**: `def article_views(views: pd.DataFrame) -> pd.DataFrame:`
* I/O 禁止（`read_csv`, `to_csv` などなし）
* 不要な `print` や **`sort_values` 禁止**（別手段でソート）

---

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  著者が自分の書いた記事を **少なくとも 1 回** 閲覧した著者の ID を求める。
  `author_id = viewer_id` となる行が「自分の記事を閲覧した」ことを意味する。

* 入力 DF: `{{INPUT_DATAFRAMES}}`

  ```text
  views: pd.DataFrame

  columns:
    - article_id : int
    - author_id  : int
    - viewer_id  : int
    - view_date  : datetime64[ns] / object（日付相当）
  ```

* 出力: `{{OUTPUT_COLUMNS_AND_RULES}}`

  ```text
  columns:
    - id : int  # 自分の記事を閲覧した著者 ID

  仕様:
    - 各著者は 1 回のみ（重複なし）
    - id 昇順ソート
  ```

---

## 2) 実装（指定シグネチャ厳守）

> この問題は **グループ集計不要** で、
> 「条件フィルタ → 著者IDのユニーク化 → ソート」が最軽量パターンです。

```python
import pandas as pd

def article_views(views: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['id']
    """
    # 著者 = 閲覧者 となる行だけを抽出（自分の記事を閲覧した行）
    self_views = views[views["author_id"] == views["viewer_id"]]

    # 著者IDのみを取り出し、ユニーク化して列名を id に変更
    out = (
        self_views[["author_id"]]
        .drop_duplicates()
        .rename(columns={"author_id": "id"})
    )

    # sort_values 禁止のため、インデックスにして sort_index で昇順ソート
    out = out.set_index("id").sort_index().reset_index()

    return out

Analyze Complexity
Runtime 281 ms
Beats 60.21%
Memory 67.53 MB
Beats 29.19%

```

* ポイント:

  * **列最小化**：`["author_id"]` のみ扱うことでメモリも処理も軽くする
  * **`drop_duplicates()`** で著者 ID を一意に
  * 明示的な `sort_values` を避け、`set_index` + `sort_index` で昇順を実現

---

## 3) アルゴリズム説明

使用 API:

1. **ブールインデックス**

   ```python
   views["author_id"] == views["viewer_id"]
   ```

   * 著者と閲覧者が同一の行のみ True
   * `views[condition]` で「自分の記事を閲覧した」ログだけを取り出す

2. **列最小化**

   ```python
   self_views[["author_id"]]
   ```

   * 不要な列を落とし、後続処理（重複削除・ソート）を軽量化

3. **重複削除 `drop_duplicates`**

   ```python
   .drop_duplicates()
   ```

   * 同じ著者が何度も自分の記事を見ていても 1 行にまとめる
   * 著者 ID のユニーク集合が得られる

4. **`rename` による列名変更**

   ```python
   .rename(columns={"author_id": "id"})
   ```

   * 問題仕様の列名 `id` に合わせる

5. **`set_index` + `sort_index` による昇順ソート**

   ```python
   out = out.set_index("id").sort_index().reset_index()
   ```

   * `id` を一度インデックスにしてから `sort_index()` でソート
   * `sort_values` を使わずに昇順を実現
   * 最後に `reset_index()` で通常列として `id` を戻す

### NULL / 重複 / 型 について

* `author_id` / `viewer_id` が NULL の場合
  → `==` 比較は `False` になり、その行は自然に除外される
  （「著者不明」や「閲覧者不明」は「自分の記事を見た」と判断できないので妥当）
* `drop_duplicates()` は完全一致行を 1 行にまとめるだけなので、
  「同じ著者 ID」が複数行あっても安全にユニーク化できる。
* 型は `int` 前提だが、文字列でもロジック自体は同じく動作する。

---

## 4) 計算量（概算）

`N` = `views` の行数とすると:

1. **ブールフィルタ**

   ```python
   views["author_id"] == views["viewer_id"]
   ```

   * 全行を 1 回見るので **O(N)**

2. **列抽出 + `drop_duplicates`**

   * 列抽出自体はビュー作成に近く、ほぼ **O(N)**（参照コピー主体）
   * `drop_duplicates` はハッシュセットベースでおおよそ **O(N)**

3. **`set_index` + `sort_index`**

   * インデックス設定: **O(N)**
   * ソート: **O(M log M)** （M は自分の記事を閲覧した著者数、通常 N より小さい）

トータルで概ね **O(N + M log M)**。
M ≪ N であればほぼ線形に近いコストで動作します。

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A["views DataFrame
  (article_id, author_id, 
  viewer_id, view_date)"] --> B[author_id = viewer_id
  でフィルタ]
  B --> C[author_id 列だけ抽出
  列最小化]
  C --> D[drop_duplicates で
  著者IDのユニーク集合]
  D --> E[列名を id に rename]
  E --> F["set_index(id) → sort_index()
  → reset_index()"]
  F --> G[出力: id 列のみの
   DataFrame]
```

この実装で、LeetCode の Pandas 版要件（シグネチャ、列名、順序、I/O 無し、`sort_values` 不使用）をすべて満たしつつ、処理も十分に軽量に保てます。

結論から言うと、そのスコアは**かなり良い部類**で、アルゴリズムとして大きな改善余地はほぼありませんが、
「pandas のオーバーヘッドを少しでも削る」という意味での**微調整**はできます 💡

---

## 1. いまの実装の位置づけ

前回お渡しした実装はだいたいこんな流れでしたね：

```python
self_views = views[views["author_id"] == views["viewer_id"]]

out = (
    self_views[["author_id"]]
    .drop_duplicates()
    .rename(columns={"author_id": "id"})
)

out = out.set_index("id").sort_index().reset_index()
```

アルゴリズムとしてはすでに最適レベルで、

* 1 回のフィルタ（O(N)）
* 著者 ID ユニーク化（O(N)）
* 著者数 M に対してソート（O(M log M)）

という感じで、これ以上 “理論オーダー” を改善する余地はありません。

LeetCode の `Runtime` / `Beats` は
**同時期に走っている他ユーザーやマシン負荷の影響でかなりブレる**ので、
281ms / 60% は普通に「十分速い」です。

ただ、**pandas の高機能メソッドを少し減らす**ことで、
オーバーヘッドを減らす = 数 ms〜十数 ms 程度縮む可能性はあります。

---

## 2. pandas を薄くしてみる改善案（おすすめ）

ポイントは：

* **DataFrame の生成・コピーを減らす**
* **Series / ndarray ベースで処理**
* `drop_duplicates` や `set_index` を避ける
* `sort_values` 禁止ルールは守る

こうすると、こんな感じにできます。

```python
import pandas as pd

def article_views(views: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['id']
    """
    # ❶ 著者 = 閲覧者 の行だけブール配列で抽出
    mask = views["author_id"].values == views["viewer_id"].values

    # ❷ 対象行の author_id を 1 次元配列として取得
    #    unique() は ndarray を返すので、そのまま扱える
    ids = views.loc[mask, "author_id"].unique()

    # ❸ id 昇順ソート（ndarray に対してそのまま in-place sort）
    ids.sort()

    # ❹ DataFrame 化（1 回だけ）
    return pd.DataFrame({"id": ids})
```

### なにが軽くなっているか

* `self_views = views[...]` という **中間 DataFrame を作っていない**
* `drop_duplicates` ではなく、`unique()` で **Series → ndarray 1 本** にしている
* `set_index` / `sort_index` / `reset_index` といった
  **インデックス操作を全部削っている**
* `sort_values` も使っていない（`ndarray.sort()`）

処理フローとしては：

1. `mask`（`bool` 配列）だけ作る → DataFrame に比べて軽い
2. `views.loc[mask, "author_id"].unique()` で **1 本の ndarray** を作る
3. その ndarray を **in-place sort**
4. 最後に 1 回だけ DataFrame を作る

ので、**中間オブジェクトの量とコピー回数がかなり減っています**。

実行時間・メモリともに、少しだけ改善する可能性が高いです。

---

## 3. メモリ 67MB / 29% について

LeetCode の「Memory Beats X%」は、

* Python のランタイム / JIT 的な Overhead
* テストケースのサイズ
* 他ユーザーのコードのメモリ特性

などが絡むので、**純粋にあなたの関数だけのメモリ使用量を正確に示しているわけではありません**。

今回の関数は

* 入力 DF `views` をそのまま利用
* 中間としては `mask`（bool 配列）と `ids`（1 次元の著者配列）
* 最後に `{"id": ids}` の 1 列データフレーム

だけなので、**ロジック的にはかなりメモリ効率も良い部類**です。

これ以上メモリを削るとしたら

* `mask` を作らずに `query` を使う（→ 逆に遅く・重くなることが多い）
* 純粋な `numpy` だけでやる（→ LeetCode 的にはやりすぎ）

など、現実にはあまり得をしない方向になるので、
**今の改善版くらいがちょうどバランスの良い落としどころ**だと思ってもらって大丈夫です。

---

## 4. まとめ

* 281ms / 60% / 67MB / 29% は **全然悪くないどころか十分合格点** ✅
* アルゴリズム（O(N + M log M)）はすでに最適クラスで、
  これ以上は **pandas のオーバーヘッド削減による微調整**の世界
* 上の `mask + unique + ndarray.sort` 版は

  * 中間 DataFrame を作らない
  * `drop_duplicates` / `set_index` / `sort_index` / `reset_index` を使わない
    という点で、**若干速く・若干メモリも軽くなる余地があります**。

もしこの改善版を試して、また Runtime / Memory の結果が出たら、それをベースにさらに細かいチューニング（例えば `values` の使い方やチェーンの切り方）も一緒に見ていきましょう 💪

エラーの原因はここです👇

```python
ids = views.loc[mask, "author_id"].unique()
ids.sort()  # ← ここで AttributeError
```

`unique()` の戻り値が **NumPy 配列ではなく `IntegerArray`（pandas の拡張配列）** になっていて、
この `IntegerArray` には `.sort()` メソッドがないため `AttributeError` が出ています。

pandas 2 系では、`Int64`（nullable int）などの拡張 dtype だと

* `Series.unique()` → `IntegerArray` を返す

という挙動になるので、私の前回の「`unique()` して `.sort()`」案がそのままだと壊れます。

---

## 対応方法①（おすすめ：NumPy の `unique` を使う）

NumPy の `np.unique` は

* ソート済みのユニーク値を返してくれる
* 戻り値は NumPy 配列（`ndarray`）なので、そのまま DataFrame にできる

ので、**そもそも `.sort()` を呼ばずに済ませる**のが一番シンプルです。

```python
import pandas as pd
import numpy as np

def article_views(views: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['id']
    """
    # 著者 = 閲覧者 の行だけブール配列で抽出
    mask = views["author_id"].values == views["viewer_id"].values

    # 対象行の author_id を取り出し、NaN を除外して numpy 配列化
    s = views.loc[mask, "author_id"].dropna()

    # ユニークかつ昇順の ID を取得（np.unique はソート済みで返す）
    ids = np.unique(s.to_numpy())

    # 仕様どおり id 列だけの DataFrame にして返す
    return pd.DataFrame({"id": ids})

Analyze Complexity
Runtime 282 ms
Beats 58.31%
Memory 66.98 MB
Beats 92.43%

```

ポイント:

* `dropna()` しておくことで、もし `author_id` に `<NA>` が混じっていても安全
* `np.unique` は「ユニーク＋昇順ソート」を一度でやってくれる
* `sort_values` は使っていないので、問題の制約も満たす

---

## 対応方法②（NumPyを使わない純Python版）

もし「NumPy使いたくない／使えない」ケースがあれば、
`set` ＋ `sorted` で同じことができます。

```python
import pandas as pd

def article_views(views: pd.DataFrame) -> pd.DataFrame:
    mask = views["author_id"].values == views["viewer_id"].values

    s = views.loc[mask, "author_id"].dropna()

    # set でユニーク化し、sorted で昇順リストに
    ids = sorted(set(s.tolist()))

    return pd.DataFrame({"id": ids})

Analyze Complexity
Runtime 262 ms
Beats 92.26%
Memory 66.62 MB
Beats 98.51%

```

こちらも

* 中間 DataFrame を作らない
* `sort_values` は使わない
* ユニーク＋昇順を満たす

という点で要件を満たします。

---

## まとめ

* エラーの直接原因は
  **`IntegerArray`（拡張配列）には `.sort()` がない**のに呼んでしまったこと。
* 対応としては

  * `np.unique` を使って「ユニーク＋ソート」を一発でやる
  * もしくは `set` + `sorted` で純 Python でやる

のどちらかに変えれば解決します。

上のどちらかに置き換えて再実行してみてください。
もしまた別のエラーや Runtime / Memory の数字が出たら、それを見ながらさらに微調整も一緒にやりましょう 💪

